# 3 Extract facial data from videos.

We use a range of libraries to extract facial data from the videos. The main library is [DeepFace](https://github.com/serengil/deepface) but we also try FER - [Facial Expression Recognition](https://github.com/justinshenk/fer).

`pip install deepface`



## 3.1 DeepFace



In [ ]:
from deepface import DeepFace
import pandas as pd
import pprint
import cv2
import os
import time
import utils


In [ ]:
videos_in = "..\\LookitLaughter.test\\"
data_out = "..\\data\\1_interim\\"

test_video = "2UWdXP.joke1.rep2.take1.Peekaboo.mp4"
test_video = "6c6MZQ.joke1.rep1.take1.ThatsNotAHat.mp4"

test_image = "peekaboo.png"
test_image2 = "mother-and-baby-in-park.jpg"

In [ ]:
#just playing around to try out different models built into deepface
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
]

result = DeepFace.analyze(img_path=test_image2, enforce_detection = False, detector_backend = backends[1])

for r in result:
    pprint.pprint(r)


In [ ]:
def createfacesdf():
    #creates a dataframe with the facial data from the videos
    cols = ['frame', 'person', "bbox.x", "bbox.y","bbox.w","bbox.h","emotion","age","gender" ] #, "allemotions","allgenders"]
    return pd.DataFrame(columns=cols)

def addfacestodf(facesdf,frameidx, facedata):
    for idx, face in enumerate(facedata):
        newrow = {'frame': frameidx,
                  'person':idx,
                  'bbox.x':face['region']['x'],
                  'bbox.y':face['region']['y'],
                  'bbox.w':face['region']['w'],
                  'bbox.h':face['region']['h'],
                  'emotion':face['dominant_emotion'],
                  'age':face['age'],
                  'gender':face['dominant_gender']}
        #print(newrow)
        facesdf.loc[len(facesdf)] = newrow
    return facesdf

In [ ]:
def getfacedataforvideo(video_path, backend = 'ssd'):
    print("Processing video: ", video_path)
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    facesdf = createfacesdf()
    frameidx = 0
    fails = []
    while success:
        try:  
            faces = DeepFace.analyze(img_path = image, 
                                    enforce_detection = True, 
                                    actions = ('age','gender','emotion'),
                                    detector_backend = backend)
        except:
            #print(f"deepface.analyse failed for frame {frame}" )
            fails.append(frameidx)
            faces = []
        if len(faces) > 0:
            facesdf = addfacestodf(facesdf,frameidx,faces)
        success,image = cap.read()
        frameidx += 1
    
    print(f"Failed to process {len(fails)}/{frameidx} ({(round(100*len(fails)/frameidx, 1))}%) frames")
    cap.release()
    return facesdf



In [ ]:
processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

In [ ]:
forceFaces = True
backend = "ssd"

for index, r in processedvideos.iterrows():
    if forceFaces or pd.isnull(r["Faces.file"]):
        facessdf = getfacedataforvideo(videos_in + r["VideoID"], backend = backend)
        stemname = os.path.splitext(r["VideoID"])[0]
        facefile =  stemname + f".faces.{backend}.csv"
        facessdf.to_csv(data_out + facefile)
        r["Faces.file"] = data_out + facefile
        r["Faces.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
        

processedvideos.to_excel(data_out + "\\processedvideos.xlsx")
processedvideos.head()

In [13]:

processedvideos.to_excel(data_out + "\\processedvideos.xlsx")
processedvideos.head()

,Index,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,...,Audio.file,Faces.when,Faces.file,LastError,Speech.file,Speech.when,Objects.file,Objects.when,Understand.file,Understand.when
0,0,2UWdXP.joke1.rep2.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,217,...,..\data\1_interim\\2UWdXP.joke1.rep2.take1.Pee...,2023-09-24 07:52:41,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2023-09-20 16:58:38,NaN,NaN,NaN,NaN
1,1,2UWdXP.joke1.rep3.take1.Peekaboo.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,152,...,..\data\1_interim\\2UWdXP.joke1.rep3.take1.Pee...,2023-09-24 07:54:38,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,NaN,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2023-09-20 16:58:39,NaN,NaN,NaN,NaN
2,2,2UWdXP.joke2.rep1.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,95,...,..\data\1_interim\\2UWdXP.joke2.rep1.take1.Nom...,2023-09-24 07:55:58,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
3,3,2UWdXP.joke2.rep2.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,97,...,..\data\1_interim\\2UWdXP.joke2.rep2.take1.Nom...,2023-09-24 07:56:57,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2023-09-20 16:58:40,NaN,NaN,NaN,NaN
4,4,2UWdXP.joke2.rep3.take1.NomNomNom.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,133,...,..\data\1_interim\\2UWdXP.joke2.rep3.take1.Nom...,2023-09-24 07:59:00,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,NaN,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2023-09-20 16:58:48,NaN,NaN,NaN,NaN
